In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from datetime import timedelta
from sklearn.metrics import mean_squared_error

# data load

In [59]:
df = pd.read_csv('../input/appliances-energy-prediction/KAG_energydata_complete.csv')
df.head(5)

10분 마다 하나의 데이터 포인트가 있다.

In [4]:
df.shape

* Date : year-month-day hour:minute:second

* Appliances : energy use in Wh

* Lights : energy use of light fixtures in the house in Wh

* T1 : Temperature in kitchen(Celsius)
* RH_1 : Humidity in kitchen(%)
* T2 : emperature in living room
* RH_2 : Humidity in living room
* T3 : Temperature in laundry room
* RH_3 : Humidity in laundry room
* T4 : Temperature in office room
* RH_4 : Humidity in office room
* T5 : Temperature in bathroom
* RH_5 : Humidity in bathroom
* T6 : Temperature outside the building
* RH_6 : Humidity outside the building
* T7 : Temperature in ironing room
* RH_7 : Humidity in ironing room
* T8 : Temperature in teenager room 
* RH_8 : Humidity in teenager room
* T9 : Temperature in parents room
* RH_9 : Humidity in parents room

* T_out : Temperature outside (from Chievres weather station)
* Press_mm_hg : Pressure (from Chievres weather station)
* RH_out : Humidity outside (from Chievres weather station)
* Windspeed : (from Chievres weather station) (m/s)
* Visibility : (from Chievres weather station) (km)
* Tdewpoint : (from Chievres weather station) (Â°C)
* Rv1 : Random variable 1, nondimensional
* Rv2 : Random variable 2, nondimensional


# EDA

In [63]:
from sklearn.model_selection import train_test_split

# 75% of the data is usedfor the training of the models and the rest is used for testing
train, test = train_test_split(df,test_size=0.25,random_state=40)

In [64]:
# Divide the columns based on type for clear column management 

col_time=["date"]

col_temp = ["T1","T2","T3","T4","T5","T6","T7","T8","T9"]

col_hum = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_6","RH_7","RH_8","RH_9"]

col_weather = ["T_out", "Tdewpoint","RH_out","Press_mm_hg",
                "Windspeed","Visibility"] 
col_light = ["lights"]

col_randoms = ["rv1", "rv2"]

col_target = ["Appliances"]

In [65]:
# Seperate dependent and independent variables 
feature_vars = train[ col_time + col_temp + col_hum + col_weather + col_light + col_randoms ]
target_vars = train[col_target]

In [66]:
# Check the distribution of values in lights column
feature_vars.lights.value_counts()

In [46]:
# Histogram of all the features to understand the distribution
feature_vars.hist(bins = 20 , figsize= (12,16)) ;

대부분은 괜찮아보이는데, RH_6, RH_out, Visibility, Windspeed 변수는 불균형한 분포를 보이고 있다. 따라서 스케일링 과정이 필요하다.

In [70]:
# Use the weather , temperature , applainces and random column to see the correlation
train_corr = train[col_temp + col_hum + col_weather +col_target+col_randoms]
corr = train_corr.corr()
# Mask the repeated values
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
  
f, ax = plt.subplots(figsize=(16, 14))
#Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr, annot=True, fmt=".2f" , mask=mask,)
    #Apply xticks
plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
plt.show()

상관관계 높은 상위 3개
* Rv1 & Rv2
* T6 & T_out
* T7 & T9

date 변수는 string으로 datetime 혹은 pandas timestamp 형식으로 변환해야함.

In [5]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'][0], list(df['date'])[-1])

데이터 세트가 거의 5달의 데이터를 가지고 있는데, 한 달 정도의 데이터만 고려하자.

In [6]:
data = df[(df['date'] > '2016-02-01 00:00:00') & (df['date'] < '2016-02-28 23:59:59')]
data = data.reset_index(drop=True)

In [7]:
# Number of rows in filtered dataframe
len(data)

In [8]:
data = data[['date','Appliances', 'T1', 'RH_1']]

plt.rcParams["figure.figsize"] = (18,5)
plt.plot(data['date'], data['Appliances'])
plt.xticks(rotation=45)
plt.show()

데이터 변동이 너무 많다. 1시간마다 모든 데이터 포인트의 평균값을 취하여 데이터를 다시 샘플링해서 플롯을 더 부드럽게 만들수 있다.

In [9]:
# To resample the data we need to make timestamp column as the index of the dataframe
data = data.set_index('date')

In [10]:
# Resampling
data = data.resample("60T").mean()
print(f"Length after resampling: {len(data)}")
data.head()

In [11]:
# 위의 그래프와 비교해서 좀 더 부드러워진 것을 확인할 수 있다.
plt.rcParams["figure.figsize"] = (18,5)
plt.plot(data['Appliances'])
plt.xticks(rotation=45)
plt.show()

데이터 세트의 총 길이는 672. 648개의 값을 훈련 데이터로 사용. 일단 훈련되면 모델은 미래의 24개 값(24시간의 데이터)을 예측한다.

In [12]:
train_data = data.head(n=648)
test_data = data.tail(n=24)

In [13]:
plt.rcParams["figure.figsize"] = (18,5)
plt.plot(train_data['Appliances'], label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.xticks(rotation=45)
plt.legend()
plt.show()

그래프에서 주황색 부분이 우리가 예측하려는 부분이다.

In [14]:
# Constant variables
forecast_horizon = 24
train_size = 648

# Stationary test

In [15]:
from statsmodels.tsa.stattools import adfuller

In [16]:
result = adfuller(train_data['Appliances'].values)
print('ADF Statistic: %f' % result[0])

데이터는 거의 고정적. 음수 값이 클수록 데이터가 더 안정적임을 의미한다. 고정되지 않은 데이터 세트는 높은 ADF 통계량을 갖습니다. 데이터 세트가 거의 고정되어 있으므로 작은 차분 값을 사용하여 더 고정적으로 만들 수 있습니다.

# Partial Auto Correlation Plot
부분 자기 상관 플롯은 데이터 포인트의 값이 이전 데이터 포인트의 값에 얼마나 의존하는지에 대한 정보를 제공한다.

In [17]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(train_data['Appliances'])
plt.show()

위의 플롯을 보면 처음 두 값 사이에만 강한 상관 관계가 있다. 자동 회귀 모델(ARIMA, SARIMA, Auto-ARIMA)의 경우 AR 매개변수 2가 좋다. 우리의 데이터는 거의 고정되어 있으므로 1의 작은 차이가 맞을 것이다.

# Scaling Data

In [18]:
# Scaling. We will use standard scaler
scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(train_data['Appliances'].values.reshape(-1, 1))

# 1. ARIMA

In [19]:
from statsmodels.tsa.arima.model import ARIMA

In [20]:
# Values for 'order' parameter are selected after hyper parameter tuning. These values give the lowest MSE of all the values 
# we tried.
model = ARIMA(scaled_train_data, order=(2,1,0))
model = model.fit()

In [21]:
# Forecast values for next 24 hours
forecasts = model.predict(start=scaled_train_data.size, end=scaled_train_data.size+forecast_horizon-1, dynamic=True)
forecasts = scaler.inverse_transform(forecasts.reshape(-1, 1))

In [22]:
# Plot
plt.rcParams["figure.figsize"] = (18,5)
plt.plot(train_data['Appliances'], label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(test_data.index, forecasts, label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Plotting only a part of training data
plt.plot(train_data['Appliances'].tail(150), label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(test_data.index, forecasts, label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

In [23]:
mse = mean_squared_error(test_data['Appliances'].values, forecasts)
mse

MSE가 나쁘지 않지만, 예측 결과는 그렇게 좋아 보이지 않는다.

# 2. Prophet

In [24]:
# !pip install fbprophet

In [25]:
from prophet import Prophet
import pandas as pd

In [26]:
train_data.head() 

Prophet은 입력 데이터가 2개의 열을 포함하는 데이터 프레임이어야 한다. datetime 형식의 시간 값을 포함하는 'ds' 열과 훈련 데이터를 포함하는 'y' 열.

In [27]:
prophet_train_data = train_data.reset_index()
prophet_train_data.columns = ['ds', 'y', 'T1', 'RH_1']

prophet_train_data.head()

In [28]:
# fitting
model = Prophet()
model.fit(prophet_train_data)

In [29]:
# Create a forecast dataframe containing only dates
forecast_df = test_data.reset_index()
forecast_df = forecast_df[['date']]
forecast_df.columns = ['ds']
forecast_df.head()

In [30]:
# Forecast values
forecast_values = model.predict(forecast_df)
forecast_values = forecast_values[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [31]:
# Plot
plt.rcParams["figure.figsize"] = (18,5)
plt.plot(train_data['Appliances'], label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(test_data.index, forecast_values['yhat'], label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Plotting only a part of training data
plt.plot(train_data['Appliances'].tail(150), label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(forecast_values['ds'], forecast_values['yhat'], label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

In [32]:
mse = mean_squared_error(test_data['Appliances'].values, forecast_values['yhat'].values)
mse

ARIMA 결과와 같은 직선 대신 현재 예측에 약간의 변동이 있다. 그러나 MSE는 비교적 높다.

# 3. SARIMA

In [33]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [34]:
my_order = (2, 1, 0)
my_seasonal_order = (2, 1, 0, 5)

In [35]:
# define model & fitting
model = SARIMAX(scaled_train_data, order=my_order, seasonal_order=my_seasonal_order)
model_fit = model.fit(disp=0)

In [36]:
# Forecast total vaccinations for next 90 days
forecasts = model_fit.predict(start=scaled_train_data.size, end=scaled_train_data.size+forecast_horizon-1)
forecasts = scaler.inverse_transform(forecasts.reshape(-1, 1))

In [37]:
# Plot
plt.rcParams["figure.figsize"] = (18,5)
plt.plot(train_data['Appliances'], label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(test_data.index, forecasts, label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Plotting only a part of training data
plt.plot(train_data['Appliances'].tail(200), label='Train data')
plt.plot(test_data['Appliances'], label='Test data')
plt.plot(test_data.index, forecasts, label='Forecasts')
plt.legend()
plt.xticks(rotation=45)
plt.show()

In [38]:
mse = mean_squared_error(test_data['Appliances'].values, forecasts)
mse

모델이 하향 추세를 예측할 수 있다. 또한 MSE도 좋은 값을 갖는다.